# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import config
from config import g_key
# configure gmaps api key
gmaps.configure(api_key=g_key)

from matplotlib import figure

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#csv is named citydata.csv
city_data_vaca_df = pd.read_csv ("citydata.csv")

#double check correct csv
city_data_vaca_df.head(100)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Caravelas,-17.7125,-39.2481,84.47,56,51,9.71,BR,1613224919
1,1,Mataura,-46.1927,168.8643,57.00,74,100,1.01,NZ,1613224919
2,2,Ushuaia,-54.8000,-68.3000,46.40,71,90,11.50,AR,1613224648
3,3,Vaini,-21.2000,-175.2000,78.80,89,20,5.75,TO,1613224920
4,4,Bluff,-46.6000,168.3333,57.00,74,100,1.01,NZ,1613224920
...,...,...,...,...,...,...,...,...,...,...
95,95,Forsytheganj,52.6333,29.7333,12.24,95,100,17.78,BY,1613224928
96,96,Castro,-24.7911,-50.0119,74.23,71,3,5.82,BR,1613224928
97,97,Faya,18.3851,42.4509,68.00,49,20,11.50,SA,1613224928
98,98,Beringovskiy,63.0500,179.3167,-5.15,91,100,23.06,RU,1613224928


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#params = {"type" : "hotel",
         # "keyword" : "hotel",
         # "radius" : 5000,
         # "key" : g_key}

In [5]:
#store latitude and longitude as locations
locations = city_data_vaca_df[["Lat", "Lng"]].astype(float)

#will be used as the weight for the heatmap
humidity = city_data_vaca_df["Humidity"].astype(float)

#plot heatmap
heatmap_fig = gmaps.figure()

#create heat leyer
humidity_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False
                                    , max_intensity = 10, point_radius = 1)
#add layer
heatmap_fig.add_layer(humidity_layer)

#display
heatmap_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#create new data frame with ideal weather conditions & drop na
ideal_weather_df = city_data_vaca_df.loc[(city_data_vaca_df["Max Temp"] > 65) & (city_data_vaca_df["Max Temp"] < 80)].dropna()

#check to see filter passed
ideal_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Vaini,-21.2000,-175.2000,78.80,89,20,5.75,TO,1613224920
9,9,Yulara,-25.2406,130.9889,73.40,68,90,8.05,AU,1613224920
10,10,Rikitea,-23.1203,-134.9692,79.14,77,0,14.74,PF,1613224920
12,12,Atuona,-9.8000,-139.0333,78.37,78,6,13.35,PF,1613224920
22,22,Geraldton,-28.7667,114.6000,73.40,88,40,4.61,AU,1613224921


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

#add in hotel name column to the data frame
hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = ""
hotel_df







,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,Vaini,-21.2000,-175.2000,78.80,89,20,5.75,TO,1613224920,
9,9,Yulara,-25.2406,130.9889,73.40,68,90,8.05,AU,1613224920,
10,10,Rikitea,-23.1203,-134.9692,79.14,77,0,14.74,PF,1613224920,
12,12,Atuona,-9.8000,-139.0333,78.37,78,6,13.35,PF,1613224920,
22,22,Geraldton,-28.7667,114.6000,73.40,88,40,4.61,AU,1613224921,
...,...,...,...,...,...,...,...,...,...,...,...
521,521,San Javier,37.8063,-0.8374,71.60,26,0,9.22,ES,1613224952,
523,523,Thai Binh,20.4500,106.3333,66.20,77,0,4.61,VN,1613224953,
526,526,Likasi,-10.9814,26.7333,76.08,84,4,5.35,CD,1613224953,
528,528,Sibolga,1.7427,98.7792,79.50,74,65,2.53,ID,1613224953,


In [8]:
#parameters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# run a request using our params dictionary
response = requests.get(base_url, params=params)

print(response)

<Response [200]>


In [12]:

#for loop to get the hotel names for each city, uses latitude and longitude coordinates to get hotels
for index, row in hotel_df.iterrows():
    #get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    #make API hit
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    #save the hotel to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    #skip if no hotel
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 3: Vaini.
Closest hotel in Vaini is Paradise First Hotel.
------------
Retrieving Results for Index 9: Yulara.
Closest hotel in Yulara is Sails in the Desert.
------------
Retrieving Results for Index 10: Rikitea.
Closest hotel in Rikitea is Pension Maro'i.
------------
Retrieving Results for Index 12: Atuona.
Closest hotel in Atuona is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 22: Geraldton.
Closest hotel in Geraldton is Ocean Centre Hotel.
------------
Retrieving Results for Index 26: Busselton.
Closest hotel in Busselton is The Sebel Busselton.
------------
Retrieving Results for Index 33: Sal Rei.
Closest hotel in Sal Rei is Hotel Riu Karamboa.
------------
Retrieving Results for Index 34: Bintulu.
Closest hotel in Bintulu is Fairfield by Marriott Bintulu Paragon.
------------
Retrieving Results for Index 36: Port Macquarie.
Closest hotel in Port Macquarie is Sails Port Macquarie by Rydges.
------------
Retrieving Results fo

Missing field/result... skipping.
------------
Retrieving Results for Index 290: Canutama.
Closest hotel in Canutama is Hotel Jhennifer.
------------
Retrieving Results for Index 294: Basco.
Closest hotel in Basco is Amboy Hometel.
------------
Retrieving Results for Index 303: Tadine.
Missing field/result... skipping.
------------
Retrieving Results for Index 312: Nānākuli.
Closest hotel in Nānākuli is Paradise Cove Luau.
------------
Retrieving Results for Index 316: Marsh Harbour.
Closest hotel in Marsh Harbour is Abaco Beach Resort.
------------
Retrieving Results for Index 318: Nagar.
Closest hotel in Nagar is Hotel Park Palace.
------------
Retrieving Results for Index 319: Nchelenge.
Closest hotel in Nchelenge is Relaxin Executive Lodge.
------------
Retrieving Results for Index 322: Alofi.
Closest hotel in Alofi is Taloa Heights.
------------
Retrieving Results for Index 326: Hūn.
Closest hotel in Hūn is Alrwasi Hotel.
------------
Retrieving Results for Index 327: Jagalūr.
Clo

In [13]:
#export hotel into csv and drop na
hotel_df.dropna()
hotel_df.to_csv('../output_data/hotel_export.csv')

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:

# Add marker layer ontop of heat map
#define layers
map_markers = gmaps.marker_layer(locations)

#add layers onto map
heatmap_fig.add_layer(map_markers)

#display map
heatmap_fig


Figure(layout=FigureLayout(height='420px'))